![](http://i.imgur.com/Fst3Sau.png)

# Quaternions

Unit quaternions provide a convenient mathematical notation for representing orientations and rotations of objects in three dimensions. Compared to Euler angles they are simpler to compose and avoid the problem of gimbal lock. Compared to rotation matrices they are more compact, more numerically stable, and more efficient.

Quaternions are composed of a real component (w) and an imaginary component (x,y,z) which has 3 elements. Thus quaternions use 4 variables to represent 3 rotational elements (think Euler: roll, pitch, and yaw) which means they have a redundant parameter. This redundancy allows them to avoid issues seen in Euler. The only problem with them is you have to be able to visualize 3D rotations in 4D space ... good luck! Thus, for humans to visualize them, we typically transform them back to Euler angles. However, the issues we see in Euler angles (gimbal lock) can reappear when we transform them. Note, the gimbal lock only is an issue in Euler angles, quaternions are fine, it is just the transformation to Euler that can have an issue.

In [1]:
from __future__ import print_function, division
from squaternion import Quaternion, euler2quat, quat2euler, quatNorm
from math import pi
from math import sqrt, atan2, asin, pi
from math import degrees as rad2deg

### Rigid Bodies Rotations

A rigid body can be rotated angle $\mu$ about an arbitrary moving/fixed axis ($\hat e$) in space by:

$$
q_{x,y,z} = \hat e \sin( \frac{\mu}{2} ) \\
q_r = \cos(\frac{\mu}{2} )
$$

Quaternion multiplication ($\otimes$) is:

$$
q \otimes p =
\begin{bmatrix}
    a & -b & -c & -d \\
    b &  a & -d &  c \\
    c &  d &  a & -b \\
    d & -c &  b &  a \\
\end{bmatrix} \cdot p = Q \cdot p \\
$$

Quaternion differential equation:

$$
\dot q = \frac{1}{2} q \otimes w \\
w = \begin{bmatrix} 0 & \omega_b \end{bmatrix}^T \\
\dot q = \frac{1}{2} W q \\
W =
\begin{bmatrix}
    0   & -w_x & -w_y & -w_z \\
    w_x & 0    & w_z  & -w_y \\
    w_y & -w_z & 0    & w_x \\
    w_z & w_y  & -w_x & 0
\end{bmatrix}
$$

# Simple Quaternions

Generally I don't need all of the capabilities (or complexity) of quaternion math libraries. Basically I just need a way to convert between Euler and Quaternion representations and have a nice way to print them out.

[squaterion](https://pypi.org/project/squaternion/) is a quaterion library. You can install it with:

    pip2 install squaternion
    pip3 install squaternion
    
The quaternion is a simple `namedtuple` so it is fast to create and destroy the datatype.

```python
Quaternion = namedtuple('Quaternion', 'w x y z')

euler2quat(roll, pitch, yaw) => Quaternion

quat2euler(w, x, y, z) => tuple(roll, pitch, yaw)

quatNorm(w, x, y, z) => Quaternion
```

In [11]:
# create a simple quaterion with no rotation
q = euler2quat(0,0,0)
w,x,y,z = q
print(q)
print(w, x, y, z)

Quaternion(w=1.0, x=0.0, y=0.0, z=0.0)
1.0 0.0 0.0 0.0


In [10]:
# let's make up some euler angles (roll, pitch, yaw)
euler = (90,45,180,)
print('euler rads:', euler)
# print('euler degs:', [x*180/pi for x in euler])

# let's create a quaternion
print('-'*40)
q = euler2quat(*euler, degrees=True)
print(q)

# let's double check euler => quaternion => euler worked
e = quat2euler(*q, degrees=True)

print('-'*40)
print("euler angles:", e)

# if you look at the answers they are the same except for some small rounding errors

euler rads: (90, 45, 180)
----------------------------------------
Quaternion(w=0.2705980500730985, x=-0.27059805007309845, y=0.6532814824381882, z=0.6532814824381883)
----------------------------------------
euler angles: (89.99999999999997, 44.99999999999999, 180.0)


# Test

![](https://upload.wikimedia.org/wikipedia/en/thumb/3/30/Plane_with_ENU_embedded_axes.svg/425px-Plane_with_ENU_embedded_axes.svg.png)

So let's do a simple test of iterating over a bunch of Euler angles, convert them to a quaternion and then back to Euler and see if we get the same answer.

    euler => quaternion => euler

Now, Euler angles have a sinularity around the following locations:

- Roll:  [-$\pi$, $\pi$]
- Pitch: [-$\pi$/2, $\pi$/2]
- Yaw:   [-$\pi$, $\pi$]

Anything outside of these Euler angle ranges will not work, unless you take special percausions. For my applications, this isn't an issue.

So let's run through a range of valid Euler angles and do the transforms and see if we have an issue. Note, because of small rounding errors, we check if the answers are the same within 0.001 degrees. If **no errors print out** and all you see is **Done**, then everything went well.

In [14]:
# run though and make sure there are no errors
# https://en.wikipedia.org/wiki/Euler_angles#Conventions_2
# valid ranges:
# asin: [-pi/2, pi/2]
# cos: [0, pi]
# atan2: [-pi,pi]
#---------------------
# valid euler angles, meaning no gimbal lock issues
# roll: [-pi,pi]
# pitch: [-pi/2, pi/2]
# yaw: [-pi,pi]
delta = 10
for i in range(-179, 180, delta):
    for j in range(-89, 90, delta):
        for k in range(-179,180, delta):
            q = euler2quat(i,j,k, degrees=True)  # euler => quat
            e = quat2euler(*q, degrees=True)     # quat => euler
            for a, b in zip((i,j,k,), e):
                if abs(a - b) > 0.001:  # are the answers within 0.001 degrees?
                    print('-'*40)
                    print('Error')
                    print(i,j,k, '==', e)
                    print(q)
print("Done")

Done


## References

- [Wikipedia Convert Between Quaternions and Euler Angles](https://en.wikipedia.org/wiki/Conversion_between_quaternions_and_Euler_angles)
- [Wikipedia Euler Angle Definitions](https://en.wikipedia.org/wiki/Euler_angles#Conventions_2)
- [Wikipedia Gimbal Lock](https://en.wikipedia.org/wiki/Gimbal_lock)